### Orthorectify GOES-R ABI images that correspond with specific ASTER observation times

Use latitude/longitude bounds to specify the area of interest, download a DEM for that area and use that DEM in the orthorectifiation.

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import os
import goes_ortho
import glob
import aster_utils

In [2]:
def nearest(items, pivot):
    # https://stackoverflow.com/questions/32237862/find-the-closest-date-to-a-given-date
    return min(items, key=lambda x: abs(x - pivot))

Specify directories for inputs and outputs

In [3]:
# directory containing ASTER L1T geotiffs, we'll grab the datetime of each from their filenames
aster_directory = "/storage/spestana/ASTER/AST_L1T/geotiff/T/T_band14_Tuolumne-and-CUES"

# the top level directory that contains all the GOES-16 ABI files we want to search
goes_directory = "/storage/GOES/goes16"

# the top level directory that we will output all orthorectified GOES images to
# folder structure within will mirror our source goes_directory
output_directory = "/storage/GOES/orthorectified"

# filepath and filename to save the DEM to
dem_filepath = 'dem/tuolumne_cues_dem.tif'
# latitude and longitude bounds for the DEM
dem_bounds = (-120, 36.8, -118.25, 38.5)

print('\n\nThis script will search for ASTER geotiff files within:\n\t{}\nand for each one, will find a corresponding GOES ABI image within:\n\t{}'.format(aster_directory, goes_directory))
print('Each corresponding GOES observation will be orthorectified and cropped to the bounds of this DEM:\n\t{}\n\t{}'.format(dem_filepath,dem_bounds))
print('And the resulting orthorectified and cropped GOES ABI image will be output to:\n\t{}'.format(output_directory))
#input("\n\t\tIf this is correct, press Enter to continue...\n\t\tIf this is NOT correct, press Ctrl+C to stop the script and exit.")




This script will search for ASTER geotiff files within:
	/storage/spestana/ASTER/AST_L1T/geotiff/T/T_band14_Tuolumne-and-CUES
and for each one, will find a corresponding GOES ABI image within:
	/storage/GOES/goes16
Each corresponding GOES observation will be orthorectified and cropped to the bounds of this DEM:
	dem/tuolumne_cues_dem.tif
	(-120, 36.8, -118.25, 38.5)
And the resulting orthorectified and cropped GOES ABI image will be output to:
	/storage/GOES/orthorectified


#### Download a DEM

In [4]:
# download DEM (make sure to convert to GRS80 ellipsoid model GOES ABI fixed grid uses)
goes_ortho.get_dem(demtype='SRTMGL3', bounds=dem_bounds, out_fn=dem_filepath, proj='+proj=lonlat +datum=GRS80') 

/home/spestana/opt/anaconda3/envs/goes-linux/bin/gdalwarp -r cubic -co COMPRESS=LZW -co TILED=YES -co BIGTIFF=IF_SAFER -tr 30 30 -t_srs '+proj=lonlat +datum=GRS80' dem/tuolumne_cues_dem.tif dem/tuolumne_cues_dem_proj.tif
/home/spestana/opt/anaconda3/envs/goes-linux/bin/gdalwarp -r cubic -co COMPRESS=LZW -co TILED=YES -co BIGTIFF=IF_SAFER -tr 30 30 -t_srs '+proj=lonlat +datum=GRS80' dem/tuolumne_cues_dem.tif dem/tuolumne_cues_dem_proj.tif


Child returned 1


'dem/tuolumne_cues_dem_proj.tif'

#### Get the times of all our ASTER observations (in UTC)

In [5]:
aster_df = aster_utils.aster_timestamps(aster_directory, ext='tif')

#### Find GOES ABI images that are closest to these ASTER observations and orthorectify

In [6]:
aster_counter = 0
# for every ASTER datetime (in UTC)
for aster_datetime_UTC in aster_df.timestampUTC:
    # count
    aster_counter += 1
    print('\n\n File {} of {}'.format(aster_counter, aster_df.shape[0]))
    print('\nFor ASTER observation at {}'.format(aster_datetime_UTC))
    # find the GOES subdirectory for the corresponding year-month-day and hour
    goes_subdir = r"/{year}/{month}/{day}/{product}/{hour}/{channel}/".format(
                            goes_directory=goes_directory, 
                            year=aster_datetime_UTC.strftime('%Y'), 
                            month=aster_datetime_UTC.strftime('%m'), 
                            day=aster_datetime_UTC.day, 
                            product='ABI-L1b-RadC', 
                            hour=aster_datetime_UTC.strftime('%H'), 
                            channel='C14')
    # now within this subdirectory, the same hour of this ASTER observation
    print('\nSearching for GOES ABI imagery within:\n{}{}'.format(goes_directory,goes_subdir))
    # get the filenames of each GOES ABI image in this subdirectory
    goes_files = glob.glob(goes_directory+goes_subdir+'*.nc')
    goes_datetimes_UTC_list = []
    goes_files_list = []
    
    # create an empty dictionary we'll fill with filenames and timestamps for each subdirectory we search
    goes_dict = {}
    
    for this_goes_file in goes_files:
        this_goes_filename = this_goes_file.split('/')[-1]
        #print('\t{}'.format(this_goes_filename))
        
        # parse the timstamp in the filename 
        this_goes_datetime_UTC = this_goes_filename.split('_')[-1].split('.')[0][1:-1]
        this_goes_datetime_UTC = pd.to_datetime(this_goes_datetime_UTC, format="%Y%j%H%M%S")
        this_goes_datetime_UTC = pd.Timestamp(this_goes_datetime_UTC, tz='UTC')
        #print('\t{}'.format(this_goes_datetime_UTC))
        
        # add these to our dictionary, use the date as the key
        goes_dict[this_goes_datetime_UTC] = {}
        goes_dict[this_goes_datetime_UTC]['filepath'] = this_goes_file
        
        
    # now find the one closest to our ASTER observation
    nearest_goes_datetime_UTC = nearest(list(goes_dict.keys()), aster_datetime_UTC)
    #print('\t{} -- {}'.format(aster_datetime_UTC, nearest_goes_datetime_UTC))
    nearest_goes_filepath = goes_dict[nearest_goes_datetime_UTC]['filepath']
    #print(goes_dict[nearest_goes_datetime_UTC]['filepath'])
    print('\n\tFound nearest GOES ABI image:\n\t\tASTER datetime:\t{}\n\t\tGOES datetime:\t{}\n\t\tGOES filepath:\t{}'.format(
            aster_datetime_UTC, nearest_goes_datetime_UTC, nearest_goes_filepath))
    
    
    
    # create the output directory if it does not already exist
    output_subdir = r"{}{}".format(output_directory,goes_subdir)
    print('\n\tPreparing to output files to:\n\t{}'.format(output_subdir))
    if not os.path.exists(output_subdir):
        os.makedirs(output_subdir)   
    # create a new filename for the orthorectified image
    new_file_name = nearest_goes_filepath.split('/')[-1].split('.')[0] + '_orthorectified'
    print('\n\tNew files will be called:\n\t{}.*'.format(new_file_name))
    # Generate the pixel mapping that relates GOES ABI pixels to points on the DEM surface
    pixel_map = goes_ortho.make_ortho_map(nearest_goes_filepath, dem_filepath)
    # Apply the pixel mapping to orthorectify the GOES ABI image
    ds = goes_ortho.orthorectify_abi(nearest_goes_filepath, 
                                     pixel_map,
                                     data_vars = ['Rad'],
                                     out_filename='{}{}.nc'.format(output_subdir,new_file_name))



 File 1 of 56

For ASTER observation at 2017-03-04 18:51:03+00:00

Searching for GOES ABI imagery within:
/storage/GOES/goes16/2017/03/4/ABI-L1b-RadC/18/C14/

	Found nearest GOES ABI image:
		ASTER datetime:	2017-03-04 18:51:03+00:00
		GOES datetime:	2017-03-04 18:50:31+00:00
		GOES filepath:	/storage/GOES/goes16/2017/03/4/ABI-L1b-RadC/18/C14/OR_ABI-L1b-RadC-M3C14_G16_s20170631847509_e20170631850282_c20170631850319.nc

	Preparing to output files to:
	/storage/GOES/orthorectified/2017/03/4/ABI-L1b-RadC/18/C14/

	New files will be called:
	OR_ABI-L1b-RadC-M3C14_G16_s20170631847509_e20170631850282_c20170631850319_orthorectified.*

RUNNING: make_ortho_map()

Opening GOES ABI image...

Get inputs: projection information from the ABI radiance product
...done

Opening DEM file...

Create 2D arrays of longitude and latitude from the DEM
...done

For each grid cell in the DEM, compute the corresponding ABI scan angle (x and y, radians)
...done

Create metadata dictionary about this map
...don

...done

Create metadata dictionary about this map
...done

Create pixel map dataset
<xarray.Dataset>
Dimensions:         (latitude: 2040, longitude: 2100)
Coordinates:
  * longitude       (longitude) float64 -120.0 -120.0 -120.0 ... -118.3 -118.3
  * latitude        (latitude) float64 38.5 38.5 38.5 38.5 ... 36.8 36.8 36.8
    dem_px_angle_x  (latitude, longitude) float64 -0.06658 -0.06657 ... -0.06487
    dem_px_angle_y  (latitude, longitude) float64 0.104 0.104 ... 0.1005 0.1005
Data variables:
    elevation       (latitude, longitude) float64 2.483e+03 ... 1.433e+03
Attributes:
    longitude_of_projection_origin:       -89.5
    semi_major_axis:                      6378137.0
    semi_minor_axis:                      6356752.31414
    satellite_height:                     42164160.0
    grs80_eccentricity:                   0.0818191910435
    longitude_of_projection_origin_info:  longitude of geostationary satellit...
    semi_major_axis_info:                 semi-major axis of GR

...done

Output this result to a new NetCDF file
Saving file as: /storage/GOES/orthorectified/2017/06/7/ABI-L1b-RadC/06/C14/OR_ABI-L1b-RadC-M3C14_G16_s20171580602189_e20171580604562_c20171580605006_orthorectified.nc
...done


 File 6 of 56

For ASTER observation at 2017-06-07 06:07:19+00:00

Searching for GOES ABI imagery within:
/storage/GOES/goes16/2017/06/7/ABI-L1b-RadC/06/C14/

	Found nearest GOES ABI image:
		ASTER datetime:	2017-06-07 06:07:19+00:00
		GOES datetime:	2017-06-07 06:05:00+00:00
		GOES filepath:	/storage/GOES/goes16/2017/06/7/ABI-L1b-RadC/06/C14/OR_ABI-L1b-RadC-M3C14_G16_s20171580602189_e20171580604562_c20171580605006.nc

	Preparing to output files to:
	/storage/GOES/orthorectified/2017/06/7/ABI-L1b-RadC/06/C14/

	New files will be called:
	OR_ABI-L1b-RadC-M3C14_G16_s20171580602189_e20171580604562_c20171580605006_orthorectified.*

RUNNING: make_ortho_map()

Opening GOES ABI image...

Get inputs: projection information from the ABI radiance product
...done

Opening DE

...done

Create metadata dictionary about this map
...done

Create pixel map dataset
<xarray.Dataset>
Dimensions:         (latitude: 2040, longitude: 2100)
Coordinates:
  * longitude       (longitude) float64 -120.0 -120.0 -120.0 ... -118.3 -118.3
  * latitude        (latitude) float64 38.5 38.5 38.5 38.5 ... 36.8 36.8 36.8
    dem_px_angle_x  (latitude, longitude) float64 -0.06658 -0.06657 ... -0.06487
    dem_px_angle_y  (latitude, longitude) float64 0.104 0.104 ... 0.1005 0.1005
Data variables:
    elevation       (latitude, longitude) float64 2.483e+03 ... 1.433e+03
Attributes:
    longitude_of_projection_origin:       -89.5
    semi_major_axis:                      6378137.0
    semi_minor_axis:                      6356752.31414
    satellite_height:                     42164160.0
    grs80_eccentricity:                   0.0818191910435
    longitude_of_projection_origin_info:  longitude of geostationary satellit...
    semi_major_axis_info:                 semi-major axis of GR

...done

Output this result to a new NetCDF file
Saving file as: /storage/GOES/orthorectified/2017/11/30/ABI-L1b-RadC/06/C14/OR_ABI-L1b-RadC-M3C14_G16_s20173340602193_e20173340604565_c20173340605012_orthorectified.nc
...done


 File 11 of 56

For ASTER observation at 2017-11-30 06:07:22+00:00

Searching for GOES ABI imagery within:
/storage/GOES/goes16/2017/11/30/ABI-L1b-RadC/06/C14/

	Found nearest GOES ABI image:
		ASTER datetime:	2017-11-30 06:07:22+00:00
		GOES datetime:	2017-11-30 06:05:01+00:00
		GOES filepath:	/storage/GOES/goes16/2017/11/30/ABI-L1b-RadC/06/C14/OR_ABI-L1b-RadC-M3C14_G16_s20173340602193_e20173340604565_c20173340605012.nc

	Preparing to output files to:
	/storage/GOES/orthorectified/2017/11/30/ABI-L1b-RadC/06/C14/

	New files will be called:
	OR_ABI-L1b-RadC-M3C14_G16_s20173340602193_e20173340604565_c20173340605012_orthorectified.*

RUNNING: make_ortho_map()

Opening GOES ABI image...

Get inputs: projection information from the ABI radiance product
...done

Openi

...

...done

Output this result to a new NetCDF file
Saving file as: /storage/GOES/orthorectified/2020/01/23/ABI-L1b-RadC/06/C14/OR_ABI-L1b-RadC-M6C14_G16_s20200230606132_e20200230608505_c20200230608565_orthorectified.nc
...done


 File 46 of 56

For ASTER observation at 2020-01-23 06:07:07+00:00

Searching for GOES ABI imagery within:
/storage/GOES/goes16/2020/01/23/ABI-L1b-RadC/06/C14/

	Found nearest GOES ABI image:
		ASTER datetime:	2020-01-23 06:07:07+00:00
		GOES datetime:	2020-01-23 06:08:56+00:00
		GOES filepath:	/storage/GOES/goes16/2020/01/23/ABI-L1b-RadC/06/C14/OR_ABI-L1b-RadC-M6C14_G16_s20200230606132_e20200230608505_c20200230608565.nc

	Preparing to output files to:
	/storage/GOES/orthorectified/2020/01/23/ABI-L1b-RadC/06/C14/

	New files will be called:
	OR_ABI-L1b-RadC-M6C14_G16_s20200230606132_e20200230608505_c20200230608565_orthorectified.*

RUNNING: make_ortho_map()

Opening GOES ABI image...

Get inputs: projection information from the ABI radiance product
...done

Openi

...done

Create metadata dictionary about this map
...done

Create pixel map dataset
<xarray.Dataset>
Dimensions:         (latitude: 2040, longitude: 2100)
Coordinates:
  * longitude       (longitude) float64 -120.0 -120.0 -120.0 ... -118.3 -118.3
  * latitude        (latitude) float64 38.5 38.5 38.5 38.5 ... 36.8 36.8 36.8
    dem_px_angle_x  (latitude, longitude) float64 -0.0908 -0.0908 ... -0.09048
    dem_px_angle_y  (latitude, longitude) float64 0.1019 0.1019 ... 0.09855
Data variables:
    elevation       (latitude, longitude) float64 2.483e+03 ... 1.433e+03
Attributes:
    longitude_of_projection_origin:       -75.0
    semi_major_axis:                      6378137.0
    semi_minor_axis:                      6356752.31414
    satellite_height:                     42164160.0
    grs80_eccentricity:                   0.0818191910435
    longitude_of_projection_origin_info:  longitude of geostationary satellit...
    semi_major_axis_info:                 semi-major axis of GRS 80 r

...done

Output this result to a new NetCDF file
Saving file as: /storage/GOES/orthorectified/2020/03/5/ABI-L1b-RadC/18/C14/OR_ABI-L1b-RadC-M6C14_G16_s20200651841142_e20200651843515_c20200651844058_orthorectified.nc
...done


 File 51 of 56

For ASTER observation at 2020-04-13 18:51:17+00:00

Searching for GOES ABI imagery within:
/storage/GOES/goes16/2020/04/13/ABI-L1b-RadC/18/C14/

	Found nearest GOES ABI image:
		ASTER datetime:	2020-04-13 18:51:17+00:00
		GOES datetime:	2020-04-13 18:49:00+00:00
		GOES filepath:	/storage/GOES/goes16/2020/04/13/ABI-L1b-RadC/18/C14/OR_ABI-L1b-RadC-M6C14_G16_s20201041846134_e20201041848507_c20201041849002.nc

	Preparing to output files to:
	/storage/GOES/orthorectified/2020/04/13/ABI-L1b-RadC/18/C14/

	New files will be called:
	OR_ABI-L1b-RadC-M6C14_G16_s20201041846134_e20201041848507_c20201041849002_orthorectified.*

RUNNING: make_ortho_map()

Opening GOES ABI image...

Get inputs: projection information from the ABI radiance product
...done

Openin

...done

Create metadata dictionary about this map
...done

Create pixel map dataset
<xarray.Dataset>
Dimensions:         (latitude: 2040, longitude: 2100)
Coordinates:
  * longitude       (longitude) float64 -120.0 -120.0 -120.0 ... -118.3 -118.3
  * latitude        (latitude) float64 38.5 38.5 38.5 38.5 ... 36.8 36.8 36.8
    dem_px_angle_x  (latitude, longitude) float64 -0.0908 -0.0908 ... -0.09048
    dem_px_angle_y  (latitude, longitude) float64 0.1019 0.1019 ... 0.09855
Data variables:
    elevation       (latitude, longitude) float64 2.483e+03 ... 1.433e+03
Attributes:
    longitude_of_projection_origin:       -75.0
    semi_major_axis:                      6378137.0
    semi_minor_axis:                      6356752.31414
    satellite_height:                     42164160.0
    grs80_eccentricity:                   0.0818191910435
    longitude_of_projection_origin_info:  longitude of geostationary satellit...
    semi_major_axis_info:                 semi-major axis of GRS 80 r

...done

Output this result to a new NetCDF file
Saving file as: /storage/GOES/orthorectified/2020/06/16/ABI-L1b-RadC/18/C14/OR_ABI-L1b-RadC-M6C14_G16_s20201681846154_e20201681848527_c20201681849026_orthorectified.nc
...done


 File 56 of 56

For ASTER observation at 2020-06-17 05:54:47+00:00

Searching for GOES ABI imagery within:
/storage/GOES/goes16/2020/06/17/ABI-L1b-RadC/05/C14/

	Found nearest GOES ABI image:
		ASTER datetime:	2020-06-17 05:54:47+00:00
		GOES datetime:	2020-06-17 05:54:03+00:00
		GOES filepath:	/storage/GOES/goes16/2020/06/17/ABI-L1b-RadC/05/C14/OR_ABI-L1b-RadC-M6C14_G16_s20201690551158_e20201690553531_c20201690554030.nc

	Preparing to output files to:
	/storage/GOES/orthorectified/2020/06/17/ABI-L1b-RadC/05/C14/

	New files will be called:
	OR_ABI-L1b-RadC-M6C14_G16_s20201690551158_e20201690553531_c20201690554030_orthorectified.*

RUNNING: make_ortho_map()

Opening GOES ABI image...

Get inputs: projection information from the ABI radiance product
...done

Openi

### Convert the new NetCDF files to GeoTiffs

In [7]:
# get all the files here
filepaths = glob.glob('/storage/GOES/orthorectified/**/*.nc', recursive=True)

for i, filepath in enumerate(filepaths):
    
    # only get netcdf files
    if os.path.splitext(os.path.normpath(filepath))[-1] == '.nc':
    
        print('Converting file {} of {}'.format(i+1, len(filepaths)))
        
        # get the orthorectified GOES NetCDF filepath
        netcdf_filepath = os.path.normpath(filepath)
        
        # create a geotiff for Radiance and Brightness Temperature
        for band in ['Rad', 'tb']:
            # make a filepath for the new tif file
            geotiff_filepath = '{}_{}.tif'.format(os.path.splitext(netcdf_filepath)[0],band)
            
            # using gdal in the command line here to convert
            os.system('gdal_translate -a_srs {a_srs} -of GTiff NETCDF:{netcdf_filepath}:{band} {geotiff_filepath}'.format(
                a_srs = 'EPSG:4326',
                netcdf_filepath = netcdf_filepath,
                geotiff_filepath = geotiff_filepath,
                band = band))

Converting file 1 of 49
Converting file 2 of 49
Converting file 3 of 49
Converting file 4 of 49
Converting file 5 of 49
Converting file 6 of 49
Converting file 7 of 49
Converting file 8 of 49
Converting file 9 of 49
Converting file 10 of 49
Converting file 11 of 49
Converting file 12 of 49
Converting file 13 of 49
Converting file 14 of 49
Converting file 15 of 49
Converting file 16 of 49
Converting file 17 of 49
Converting file 18 of 49
Converting file 19 of 49
Converting file 20 of 49
Converting file 21 of 49
Converting file 22 of 49
Converting file 23 of 49
Converting file 24 of 49
Converting file 25 of 49
Converting file 26 of 49
Converting file 27 of 49
Converting file 28 of 49
Converting file 29 of 49
Converting file 30 of 49
Converting file 31 of 49
Converting file 32 of 49
Converting file 33 of 49
Converting file 34 of 49
Converting file 35 of 49
Converting file 36 of 49
Converting file 37 of 49
Converting file 38 of 49
Converting file 39 of 49
Converting file 40 of 49
Convertin